Reference [Peter Norvig](https://norvig.com/lispy.html)

Let's just make a simple calculator.  We want to be able to use it like:
```
(define r 10)
(* pi (* r r))
```

```
>> program = "(begin (define r 10) (* pi (* r r)))"

>>> parse(program)
['begin', ['define', 'r', 10], ['*', 'pi', ['*', 'r', 'r']]]

>>> eval(parse(program))
314.1592653589793
```

## Type Definitions

In [1]:
Symbol = str              # Symbol is implemented as a Python str
Number = (int, float)     # Number is implemented as either a Python int or float
Atom   = (Symbol, Number) # An Atom is a Symbol or Number
List   = list             # List is implemented as a Python list
Exp    = (Atom, List)     # An expression is either an Atom or List
Env    = dict             # An environment is a mapping of {variable: value}. Dict for now; we'll expand later.

In [2]:
def tokenize(chars: str) -> list:
    """
    Convert a string of characters into a list of tokens.
    """
    return chars.replace('(', ' ( ').replace(')', ' ) ').split()

In [3]:
program = "(begin (define r 10) (* pi (* r r)))"
print(tokenize(program))

['(', 'begin', '(', 'define', 'r', '10', ')', '(', '*', 'pi', '(', '*', 'r', 'r', ')', ')', ')']


In [4]:
def parse(program: str) -> Exp:
    """
    Read a string and turn it into an Expression.
    """
    return read_from_tokens(tokenize(program))

def read_from_tokens(tokens: list) -> Exp:
    """
    Read an expression from a sequence of tokens.
    """
    if len(tokens) == 0:
        raise SyntaxError('unexpected EOF')
    
    token = tokens.pop(0)
    if token == '(':
        L = []
        while tokens[0] != ')':
            L.append(read_from_tokens(tokens))
        tokens.pop(0) # pop off ')'
        return L
    
    if token == ')':
        raise SyntaxError('unexpected )')
        
    return atom(token)

def atom(token: str) -> Atom:
    """
    Numbers remain numbers; every other token becomes a symbol.
    """
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return Symbol(token)

In [5]:
program

'(begin (define r 10) (* pi (* r r)))'

In [6]:
parse(program)

['begin', ['define', 'r', 10], ['*', 'pi', ['*', 'r', 'r']]]

## Environments

In [7]:
#*TODO: make this a class
#*TODO: consider other ways to update the global space and expand it by importing modules
import math
import operator as op

def standard_env() -> Env:
    "An environment with some Scheme standard procedures."
    env = Env()
    env.update(vars(math)) # sin, cos, sqrt, pi, ...
    env.update({
        '+':op.add, '-':op.sub, '*':op.mul, '/':op.truediv, 
        '>':op.gt, '<':op.lt, '>=':op.ge, '<=':op.le, '=':op.eq, 
        'abs':     abs,
        'append':  op.add,  
        'apply':   lambda proc, args: proc(*args),
        'begin':   lambda *x: x[-1],
        'car':     lambda x: x[0],
        'cdr':     lambda x: x[1:], 
        'cons':    lambda x,y: [x] + y,
        'eq?':     op.is_, 
        'expt':    pow,
        'equal?':  op.eq, 
        'length':  len, 
        'list':    lambda *x: List(x), 
        'list?':   lambda x: isinstance(x, List), 
        'map':     map,
        'max':     max,
        'min':     min,
        'not':     op.not_,
        'null?':   lambda x: x == [], 
        'number?': lambda x: isinstance(x, Number),  
        'print':   print,
        'procedure?': callable,
        'round':   round,
        'symbol?': lambda x: isinstance(x, Symbol),
    })
    return env

global_env = standard_env()

## Evaluation: eval

In [22]:
def eval(x: Exp, env=global_env) -> Exp:
    """
    Evaluate an expression in an environment.
    """
    if isinstance(x, Symbol):        # variable reference
        return env[x]
    if isinstance(x, Number):      # constant number
        return x                
    if x[0] == 'if':               # conditional
        (_, test, conseq, alt) = x
        result = eval(test,env)
        exp = (conseq if eval(test, env) else alt)
        return eval(exp, env)
    if x[0] == 'define':           # definition
        (_, symbol, exp) = x
        env[symbol] = eval(exp, env)
        return None
    
    # Procedure call.
    proc = eval(x[0], env)
    args = [eval(arg, env) for arg in x[1:]]
    return proc(*args)

In [23]:
eval(parse(program))

314.1592653589793

In [38]:
def run(program: str) -> Exp:
    return eval(parse(program))

In [40]:
print(program)

(begin (define r 10) (* pi (* r r)))


In [37]:
run(program)

314.1592653589793

## Interaction: REPL

In [36]:
def repl(prompt='> '):
    """
    A prompt-read-eval-print loop.
    """
    while True:
        text = input(prompt)
        if text == 'exit':
            break
        val = eval(parse(text))
        if val is not None: 
            print(unparse(val))

def unparse(exp):
    """
    Convert an expression's internal representation Python object back into a parsable string.
    """
    if isinstance(exp, List):
        return '(' + ' '.join(map(unparse, exp)) + ')' 
    else:
        return str(exp)

Try this:
```
repl()
(+ 1 2)
exit
```

In [35]:
repl()

> (+ 1 2)
text="(+ 1 2)" is exit=False
3
> exit
text="exit" is exit=True


Try this:
```
repl()
> (define r 10)
> (* pi (* r r))
314.1592653589793
> (if (> (* 11 11) 12) (* 7 6) oops)
```

In [41]:
run('(if (> 1 2) 1 0)')

0